# TimbreNet2 Train

In [1]:
import os
import random
import datetime
import numpy as np
import tensorflow as tf
from lib.model import TimbreNet_Model
from lib.specgrams_helper import SpecgramsHelper

os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.test.is_gpu_available()

True

In [2]:
# run params
time_clock = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
RUN_ID = 'ID_'+time_clock
# RUN_ID = 'ID_2020_06_22_14_54_04'
RUN_FOLDER = './run/{}'.format(RUN_ID)

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    os.mkdir(os.path.join(RUN_FOLDER, 'logs'))
    os.mkdir(os.path.join(RUN_FOLDER + '/logs', 'scalars'))

mode =  'build' 
# mode =  'load' 

# Data

In [3]:
LEARNING_RATE = 3e-5
R_LOSS_FACTOR = 10
EPOCHS = 4
PRINT_EVERY_N_BATCHES = 10
INITIAL_EPOCH = 0
SEED = 21
BATCH_SIZE = 10
NUM_TRAIN_IMAGES = 100
NUM_TEST_IMAGES  = 100

# Architecture

In [4]:
LATENT_DIM = 2
TN_VAE = TimbreNet_Model(LATENT_DIM)

if mode == 'build':
    TN_VAE.save(RUN_FOLDER)
else:
    TN_VAE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))
    
TN_VAE.encoder.summary()
TN_VAE.decoder.summary()

#Select dataset folder
list_train_ds = tf.data.Dataset.list_files('./datasets/pianoTriadDataset/audio_mini_test/*', shuffle=True, seed=SEED)#.batch(BATCH_SIZE)
list_test_ds = tf.data.Dataset.list_files('./datasets/pianoTriadDataset/audio_mini_test/*', shuffle=True, seed=SEED)

#Apply preprocess to the dataset and batch
audio_train_ds = list_train_ds.map(TN_VAE.pre_process).batch(BATCH_SIZE).repeat(EPOCHS)
audio_test_ds  =  list_test_ds.map(TN_VAE.pre_process).batch(1)#BATCH_SIZE)

TN_VAE.compile(LEARNING_RATE, R_LOSS_FACTOR)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 1024, 2 0                                            
__________________________________________________________________________________________________
encoder_conv_in (Conv2D)        (None, 128, 1024, 32 96          encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_conv_0_1 (Conv2D)       (None, 128, 1024, 32 9248        encoder_conv_in[0][0]            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 1024, 32 128         encoder_conv_0_1[0][0]           
____________________________________________________________________________________________

# Training

In [5]:
TN_VAE.train_with_generator2(     
    audio_train_ds
    , epochs = EPOCHS
    , steps_per_epoch = NUM_TRAIN_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
    , validation_data = audio_test_ds
)

Epoch 1/4
10/10 [==============================] - 19s 2s/step - loss: 4.9410 - vae_r_loss: 4.3212 - vae_kl_loss: 0.6198 - val_loss: 2.1158 - val_vae_r_loss: 2.1153 - val_vae_kl_loss: 4.4144e-04
Epoch 2/4
10/10 [==============================] - 9s 875ms/step - loss: 2.2270 - vae_r_loss: 1.7963 - vae_kl_loss: 0.4307 - val_loss: 1.5963 - val_vae_r_loss: 1.5947 - val_vae_kl_loss: 0.0016
Epoch 3/4
10/10 [==============================] - 9s 878ms/step - loss: 1.9627 - vae_r_loss: 1.5177 - vae_kl_loss: 0.4450 - val_loss: 1.4598 - val_vae_r_loss: 1.4576 - val_vae_kl_loss: 0.0023
Epoch 4/4
10/10 [==============================] - 9s 875ms/step - loss: 1.8310 - vae_r_loss: 1.4272 - vae_kl_loss: 0.4038 - val_loss: 1.3984 - val_vae_r_loss: 1.3943 - val_vae_kl_loss: 0.0041
